In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
df_train = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_test.csv")

In [3]:
xcolnams = df_train.columns.tolist()
print("check0")
df_train[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_train["ones"] = 1.
xcolnams += ["ones"]
Xs_train = df_train[xcolnams[1:]].values.T
print("check2")

check0
check1
check2


In [4]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_train[coln] = (df_train["label"] == i).astype(float)

In [5]:
xcolnams = df_test.columns.tolist()
print("check0")
df_test[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

df_test["ones"] = 1.
xcolnams += ["ones"]
Xs_test = df_test[xcolnams[1:]].values.T
print("check2")

check0
check1
check2


In [6]:
ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_test[coln] = (df_test["label"] == i).astype(float)

In [7]:
Ys_train = df_train[ycolnams + ["ones"]].values.T

In [8]:
Ys_test = df_test[ycolnams + ["ones"]].values.T

In [9]:
sys.path.append(r"C:\Users\maxim\Desktop\js\linfit")
from linfit import lay, lay_holder

In [10]:
Ys_train.shape

(11, 60000)

In [11]:
model = lay_holder([lay(784,128), lay(128,10)]) 
#model = lay_holder([lay(784,10, False)])

In [12]:
import timeit
start_time = timeit.default_timer()
NTR = 1
for j in range(NTR):
    print("Back")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=len(model.layers)-i)
        forw = model.forward(Xs_train)
        mse = np.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)

    print("Front")
    for i in range(len(model.layers)):
        model.solve_update(Xs_train, Ys_train, n=i + 1)
        forw = model.forward(Xs_train)
        #print(forw.shape)
        mse = np.mean((forw[0,:] - Ys_train[0,:])**2)
        print(mse)
elapsed = timeit.default_timer() - start_time
print(f"Took {elapsed}s")

Back
(129, 60000)
(11, 60000)
0.02540575039442626
(785, 60000)
(129, 60000)
0.042749748038969776
Front
(785, 60000)
(129, 60000)
0.042749748038969776
(129, 60000)
(11, 60000)
0.01209476244232468
Took 19.011941199999967s


In [13]:
forw = model.forward(Xs_train)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = np.argmax(forw[:-1,:], axis=0)
# print(mse)

In [14]:
sum(guessed == df_train["label"].values) / len(guessed)

0.8979

In [15]:
1/10

0.1

In [16]:
from sklearn.metrics import confusion_matrix

In [17]:
confusion_matrix(df_train["label"].values, guessed)

array([[5717,    4,   39,    8,    6,   43,   55,    4,   43,    4],
       [   2, 6480,   68,   21,   13,   17,    4,   16,  115,    6],
       [  64,   75, 5288,  133,   67,    6,  134,   43,  125,   23],
       [  17,   40,  186, 5305,   11,  225,   18,  109,  154,   66],
       [   9,   39,   58,    4, 5313,   23,   70,    8,   68,  250],
       [  98,   39,   47,  217,   31, 4529,  133,   40,  233,   54],
       [  53,   29,   49,    2,   31,   90, 5639,    3,   21,    1],
       [  27,  104,   73,   54,  116,   12,    0, 5655,   20,  204],
       [  57,  255,   91,  201,   50,  225,   67,   14, 4774,  117],
       [  50,   18,   35,   96,  250,   34,    6,  227,   59, 5174]],
      dtype=int64)

In [18]:
forw = model.forward(Xs_test)
# #print(forw.shape)
# mse = np.mean((forw[0,:] - Ys_train[0,:])**2)

guessed = np.argmax(forw[:-1,:], axis=0)
# print(mse)

In [19]:
sum(guessed == df_test["label"].values) / len(guessed)

0.8954

In [20]:
confusion_matrix(df_test["label"].values, guessed)

array([[ 951,    0,    3,    1,    0,    5,   14,    1,    5,    0],
       [   0, 1100,    5,    4,    1,    1,    3,    2,   19,    0],
       [  12,   17,  901,   23,   11,    2,   23,   10,   28,    5],
       [   3,    1,   24,  891,    3,   31,    2,   17,   29,    9],
       [   0,    5,    6,    0,  896,    0,   17,    2,   12,   44],
       [  14,    5,    5,   45,    7,  734,   19,   16,   42,    5],
       [  14,    4,   10,    0,   10,   18,  897,    0,    4,    1],
       [   3,   19,   20,   11,   17,    0,    0,  908,    3,   47],
       [  15,   27,    8,   31,   15,   36,   14,    7,  809,   12],
       [  11,    6,    2,   15,   50,    6,    3,   36,   13,  867]],
      dtype=int64)